# © Artur Czarnecki. All rights reserved.

## Runtime Integration — STATIC + DYNAMIC (end-to-end smoke tests)

This notebook validates **runtime-level integration**:

`RuntimeEngine.run()` → `PipelineFactory` → `PlannerStaticPipeline` / `PlannerDynamicPipeline`

Key rules:
- One code cell = one independent test (no shared state).
- Only real framework components (no fakes).
- Deterministic planning via `ScriptedPlanSource` to prove the runtime path reliably.


## Test RT-S1 — Runtime STATIC_PLAN smoke (deterministic plan source)

Goal:
- Run `RuntimeEngine.run()` with `STATIC_PLAN`
- Planning is deterministic via `ScriptedPlanSource([FINALIZE])`
- Assert:
  - non-empty answer
  - route.strategy is set
  - trace contains runtime `run_start` and `run_end` events


In [ ]:
import sys, os

from intergrax.runtime.nexus.pipelines.planner_static_pipeline import PlannerStaticPipeline
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))


from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.engine.runtime import RuntimeEngine

from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest

from intergrax.runtime.nexus.planning.plan_sources import PlanSpec, ScriptedPlanSource
from intergrax.runtime.nexus.planning.engine_plan_models import EngineNextStep, PlanIntent, PlannerPromptConfig
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_planner import StepPlannerConfig
from intergrax.runtime.nexus.planning.step_executor_models import StepExecutorConfig

from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager


async def _run_rt_s1() -> None:
    # 1) Deterministic plan: FINALIZE
    plan_finalize = PlanSpec(
        version="1",
        intent=PlanIntent.GENERIC,
        next_step=EngineNextStep.FINALIZE,
        reasoning_summary="Deterministic STATIC runtime test plan (FINALIZE).",
        ask_clarifying_question=False,
        clarifying_question=None,
        use_websearch=False,
        use_user_longterm_memory=False,
        use_rag=False,
        use_tools=False,
    )
    plan_source = ScriptedPlanSource([plan_finalize])

    # 2) Real LLM adapter (same pattern as previous notebooks)
    llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

    # 3) Runtime config (STATIC requires planner_prompt_config)
    config = RuntimeConfig(
        llm_adapter=llm_adapter,
        enable_rag=False,
        enable_websearch=False,
        tools_mode="off",
        
        pipeline= PlannerStaticPipeline(),

        step_planner_cfg=StepPlannerConfig(),
        step_executor_cfg=StepExecutorConfig(),

        planner_prompt_config=PlannerPromptConfig(),
        plan_loop_policy=PlanLoopPolicy(max_replans=2, max_same_plan_repeats=2),

        # Deterministic plan injection
        plan_source=plan_source,
    )

    # 4) Context + engine
    session_manager = SessionManager(storage=InMemorySessionStorage())
    ctx = RuntimeContext.build(config=config, session_manager=session_manager)
    engine = RuntimeEngine(context=ctx)

    # 5) Request
    req = RuntimeRequest(
        session_id="rt-s1-static-session",
        user_id="rt-s1-static-user",
        message="RT-S1: return a short answer confirming STATIC runtime integration works.",
        attachments=[],
    )

    # 6) Run
    answer = await engine.run(req)

    # 7) Assertions (minimal, trustworthy)
    assert answer.answer is not None and answer.answer.strip() != "", "Expected a non-empty final answer."
    assert answer.route is not None, "Expected RouteInfo to be present."
    assert isinstance(answer.route.strategy, str) and answer.route.strategy.strip() != "", "Expected route.strategy to be set."

    events = answer.trace_events or []
    assert any(e.step == "run_start" for e in events), "Missing run_start in trace_events."
    assert any(e.step == "run_end" for e in events), "Missing run_end in trace_events."


    print("RT-S1 OK")
    print("Answer length:", len(answer.answer))
    print("Route.strategy:", answer.route.strategy)
    print("Debug trace tail:", events[-3:])


await _run_rt_s1()



RT-S1 OK
Answer length: 41
Route.strategy: llm_only
Debug trace tail: [{'ts_utc': '2026-01-09T09:01:44.651809+00:00', 'level': 'info', 'component': 'engine', 'step': 'core_llm', 'message': 'Core LLM adapter returned answer.', 'data': {'used_tools_answer': False, 'adapter_return_type': 'str', 'answer_len': 41, 'answer_is_empty': False}}, {'ts_utc': '2026-01-09T09:01:44.651809+00:00', 'level': 'info', 'component': 'engine', 'step': 'persist_and_build_answer', 'message': 'Assistant answer persisted and RuntimeAnswer built.', 'data': {'session_id': 'rt-s1-static-session', 'strategy': 'llm_only', 'used_rag': False, 'used_websearch': False, 'used_tools': False}}, {'ts_utc': '2026-01-09T09:01:44.651809+00:00', 'level': 'info', 'component': 'engine', 'step': 'run_end', 'message': 'RuntimeEngine.run() finished.', 'data': {'strategy': 'llm_only', 'used_rag': False, 'used_websearch': False, 'used_tools': False, 'used_user_longterm_memory': False, 'run_id': 'run_31c39fbe45524861a2e23662e0c831d1'}

## Test RT-D1 — Runtime DYNAMIC_LOOP smoke (strict assertions, real trace)

This is a strict runtime-level test using deterministic `ScriptedPlanSource([SYNTHESIZE → FINALIZE])`.

Assertions:
- final answer is non-empty
- `run_start` and `run_end` exist in `answer.trace_events`
- exactly 2 dynamic planning iterations occurred
- iteration 0 planned `next_step == "synthesize"`
- iteration 1 planned `next_step == "finalize"`
- planner source is scripted
- second iteration includes replan context injected with reason `iteration_completed`


In [ ]:
import sys, os

from intergrax.runtime.nexus.pipelines.planner_dynamic_pipeline import PlannerDynamicPipeline
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.engine.runtime import RuntimeEngine

from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest

from intergrax.runtime.nexus.planning.plan_sources import PlanSpec, ScriptedPlanSource
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_planner import StepPlannerConfig
from intergrax.runtime.nexus.planning.step_executor_models import StepExecutorConfig

from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager


# Deterministic plan source: SYNTHESIZE -> FINALIZE
plan_0_synthesize = PlanSpec(
    version="1",
    intent=PlanIntent.GENERIC,
    next_step=EngineNextStep.SYNTHESIZE,
    reasoning_summary="Deterministic DYNAMIC runtime test plan (SYNTHESIZE).",
    ask_clarifying_question=False,
    clarifying_question=None,
    use_websearch=False,
    use_user_longterm_memory=False,
    use_rag=False,
    use_tools=False,
)

plan_1_finalize = PlanSpec(
    version="1",
    intent=PlanIntent.GENERIC,
    next_step=EngineNextStep.FINALIZE,
    reasoning_summary="Deterministic DYNAMIC runtime test plan (FINALIZE).",
    ask_clarifying_question=False,
    clarifying_question=None,
    use_websearch=False,
    use_user_longterm_memory=False,
    use_rag=False,
    use_tools=False,
)

plan_source = ScriptedPlanSource([plan_0_synthesize, plan_1_finalize])

# Real LLM adapter
llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

# Runtime config
config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",

    pipeline= PlannerDynamicPipeline(),

    step_planner_cfg=StepPlannerConfig(),
    step_executor_cfg=StepExecutorConfig(),

    plan_loop_policy=PlanLoopPolicy(max_replans=3, max_same_plan_repeats=2),

    # Deterministic plan injection
    plan_source=plan_source,
)

# Context + engine
session_manager = SessionManager(storage=InMemorySessionStorage())
ctx = RuntimeContext.build(config=config, session_manager=session_manager)
engine = RuntimeEngine(context=ctx)

# Request
req = RuntimeRequest(
    session_id="rt-d1-dynamic-session",
    user_id="rt-d1-dynamic-user",
    message="RT-D1: return a short answer confirming DYNAMIC runtime integration works.",
    attachments=[],
)

# Run
answer = await engine.run(req)

# Basic assertions
assert answer.answer is not None and answer.answer.strip() != "", "Expected a non-empty final answer."
assert answer.route is not None and isinstance(answer.route.strategy, str) and answer.route.strategy.strip() != "", "Expected route.strategy to be set."

events = answer.trace_events or []
assert any((e.component.value == "engine" and e.step == "run_start") for e in events), "Missing engine/run_start."
assert any((e.component.value == "engine" and e.step == "run_end") for e in events), "Missing engine/run_end."

# Strict: exactly 2 dynamic loop "Engine plan produced" events from plan_loop_dynamic
loop_plan_events = [
    e for e in events
    if e.get("component") == "planner"
    and e.get("step") == "plan_loop_dynamic"
    and e.get("message") == "Engine plan produced."
]

assert len(loop_plan_events) == 2, f"Expected exactly 2 dynamic planning iterations, got {len(loop_plan_events)}"

first_next = loop_plan_events[0].get("data", {}).get("next_step")
second_next = loop_plan_events[1].get("data", {}).get("next_step")

assert first_next == "synthesize", f"Expected iteration 0 next_step='synthesize', got {first_next!r}"
assert second_next == "finalize", f"Expected iteration 1 next_step='finalize', got {second_next!r}"

# Strict: ensure plan source is scripted (look at planner debug event "Engine plan produced." from engine_planner)
engine_planner_events = [
    e for e in events
    if e.get("component") == "planner"
    and e.get("step") == "engine_planner"
    and e.get("message") == "Engine plan produced."
]

assert len(engine_planner_events) >= 2, f"Expected at least 2 engine_planner plan events, got {len(engine_planner_events)}"

# We assert that at least one of them reports scripted plan source.
assert any(
    (e.get("data", {}).get("debug", {}) or {}).get("planner_source_kind") == "scripted"
    for e in engine_planner_events
), "Expected planner_source_kind == 'scripted' in engine_planner debug."

# Strict: second iteration should have replan ctx injected with reason iteration_completed
replan_injected_events = [
    e for e in events
    if e.get("component") == "planner"
    and e.get("step") == "engine_planner"
    and e.get("message") == "Replan context injected into planner prompt."
]

assert len(replan_injected_events) >= 1, "Expected replan context injection event."
assert any(
    e.get("data", {}).get("replan_reason") == "iteration_completed"
    for e in replan_injected_events
), "Expected replan_reason == 'iteration_completed' for replan injection."

print("RT-D1 STRICT OK")
print("Answer length:", len(answer.answer))
print("Route.strategy:", answer.route.strategy)
print("Loop next_step sequence:", [first_next, second_next])
print("Events tail:", events[-5:])


RT-D1 STRICT OK
Answer length: 60
Route.strategy: llm_only
Loop next_step sequence: ['synthesize', 'finalize']
Events tail: [{'ts_utc': '2026-01-09T09:19:48.734576+00:00', 'level': 'info', 'component': 'planner', 'step': 'engine_planner', 'message': 'Replan context injected into planner prompt.', 'data': {'has_replan_ctx': True, 'replan_reason': 'iteration_completed', 'replan_hash': '4a5e5b978605556a', 'replan_json_len': 191}}, {'ts_utc': '2026-01-09T09:19:48.734576+00:00', 'level': 'info', 'component': 'planner', 'step': 'engine_planner', 'message': 'Engine plan produced.', 'data': {'intent': 'generic', 'next_step': 'finalize', 'debug': {'raw_json_shape': {'type': 'object', 'keys_count': 10, 'keys_preview': ['ask_clarifying_question', 'clarifying_question', 'intent', 'next_step', 'reasoning_summary', 'use_rag', 'use_tools', 'use_user_longterm_memory', 'use_websearch', 'version']}, 'planner_json_len': 279, 'next_step_raw': 'finalize', 'capability_clamp': {'before': {'use_websearch': Fa

## Test RT-S2 — STATIC_PLAN fail-fast when planner_prompt_config is missing

Goal:
- Configure runtime for `STATIC_PLAN`
- Do NOT provide `planner_prompt_config` and do NOT provide `plan_source`
- Expect a clear exception (fail-fast), not a silent fallback


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.engine.runtime import RuntimeEngine

from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest

from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_planner import StepPlannerConfig
from intergrax.runtime.nexus.planning.step_executor_models import StepExecutorConfig

from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager


llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

# STATIC_PLAN but missing planner_prompt_config and missing plan_source -> must fail fast
config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",

    pipeline= PlannerStaticPipeline(),

    step_planner_cfg=StepPlannerConfig(),
    step_executor_cfg=StepExecutorConfig(),

    plan_loop_policy=PlanLoopPolicy(max_replans=2, max_same_plan_repeats=2),

    # Intentionally missing:
    # planner_prompt_config=None
    # plan_source=None
)

session_manager = SessionManager(storage=InMemorySessionStorage())
ctx = RuntimeContext.build(config=config, session_manager=session_manager)
engine = RuntimeEngine(context=ctx)

req = RuntimeRequest(
    session_id="rt-s2-static-failfast-session",
    user_id="rt-s2-static-failfast-user",
    message="RT-S2: this should fail fast due to missing planner prompt config.",
    attachments=[],
)

try:
    _ = await engine.run(req)
    raise AssertionError("Expected an exception, but runtime completed successfully.")
except Exception as e:
    msg = str(e)
    print("RT-S2 OK (fail-fast)")
    print("Exception type:", type(e).__name__)
    print("Exception message:", msg)

    # Keep assertion minimal but meaningful (stable contract)
    assert ("planner_prompt_config" in msg) or ("PlannerPromptConfig" in msg) or ("prompt_config" in msg), (
        "Exception message should clearly indicate missing planner prompt configuration."
    )


RT-S2 OK (fail-fast)
Exception type: RuntimeError
Exception message: PlannerStaticPipeline: config.planner_prompt_config is required.


## Test RT-D2 — DYNAMIC_LOOP fail-fast when step_executor_cfg is missing

Goal:
- Configure runtime for `DYNAMIC_LOOP`
- Provide deterministic `plan_source` (so planning does not depend on prompt config)
- Intentionally omit `step_executor_cfg`
- Expect a clear fail-fast exception (not masked by tracker/debug wiring)


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.engine.runtime import RuntimeEngine

from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest

from intergrax.runtime.nexus.planning.plan_sources import PlanSpec, ScriptedPlanSource
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_planner import StepPlannerConfig

from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager


# Deterministic plan source
plan_source = ScriptedPlanSource([
    PlanSpec(
        version="1",
        intent=PlanIntent.GENERIC,
        next_step=EngineNextStep.FINALIZE,
        reasoning_summary="Deterministic plan for DYNAMIC fail-fast test.",
        ask_clarifying_question=False,
        clarifying_question=None,
        use_websearch=False,
        use_user_longterm_memory=False,
        use_rag=False,
        use_tools=False,
    )
])


llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

# DYNAMIC_LOOP but missing step_executor_cfg -> must fail fast
config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",

    pipeline= PlannerDynamicPipeline(),

    step_planner_cfg=StepPlannerConfig(),

    # Intentionally missing:
    # step_executor_cfg=None

    plan_loop_policy=PlanLoopPolicy(max_replans=2, max_same_plan_repeats=2),
    plan_source=plan_source,
)

session_manager = SessionManager(storage=InMemorySessionStorage())
ctx = RuntimeContext.build(config=config, session_manager=session_manager)
engine = RuntimeEngine(context=ctx)

req = RuntimeRequest(
    session_id="rt-d2-dynamic-failfast-session",
    user_id="rt-d2-dynamic-failfast-user",
    message="RT-D2: this should fail fast due to missing step_executor_cfg.",
    attachments=[],
)

try:
    _ = await engine.run(req)
    raise AssertionError("Expected an exception, but runtime completed successfully.")
except Exception as e:
    msg = str(e)
    print("RT-D2 OK (fail-fast)")
    print("Exception type:", type(e).__name__)
    print("Exception message:", msg)

    assert ("step_executor_cfg" in msg) or ("StepExecutor" in msg) or ("executor" in msg), (
        "Exception message should clearly indicate missing step executor configuration."
    )


RT-D2 OK (fail-fast)
Exception type: RuntimeError
Exception message: PlannerDynamicPipeline: config.step_executor_cfg is required.
